In [ ]:
from init import init_LLM_PURE, init_LLM_STORY, init_SDXL, init_Translator, init_TTM
from models import LLM_story, SDXL, TL, TTM
from utils import ordinal, check_GPU, translate_decisions
import logging
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# Set up basic configuration for logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
check_GPU()

In [ ]:
LLM_PURE_ = init_LLM_PURE()

In [ ]:
SDXL_, DETECTOR_ = init_SDXL()

In [ ]:
language_dict = init_Translator()

In [ ]:
TTM_ = init_TTM()

In [ ]:
# def init_models():
#     LLM_PURE_ = init_LLM_PURE()
#     SDXL_, DETECTOR_ = init_SDXL
#     language_dict = init_Translator
#     TTM_ = init_TTM
#     
#     return LLM_PURE_, SDXL_, TTM_

In [ ]:
# Current memory usage
import torch
allocated_memory = torch.cuda.memory_allocated(0) / 1024**2  # Convert to MB
print(f"  Allocated Memory: {allocated_memory:.2f} MB")

In [ ]:
def start_gen(Params, LLM_PURE_): # after configuration page, while reading the prolog
    #update Params
    keys_to_update = ["main_name", "gender"]
    for key in keys_to_update:
        if key in Params:
            Params[key] = TL(Params[key], from_code=Params["language"])
    
    LLM_STORY_ = init_LLM_STORY(LLM_PURE_, Params)
    LLM_STORY_, chapter, text = LLM_story(LLM_STORY_, "Write now only the 1st chapter!", "story", LLM_PURE_) # type: ignore
    LLM_STORY_, decisions = LLM_story(LLM_STORY_, f"Create now {Params['num_decisions']} decisions how the story could continue.", "decisions", LLM_PURE_) # type: ignore
    
    image_path = SDXL(SDXL_, DETECTOR_, LLM_PURE_, text, chap=1, decision=1)
    sound_path = TTM(TTM_, LLM_PURE_, text, chap=1, decision=1, use_ai_music=Params["use_ai_music"])
    
    language = Params["language"]
    Params["prev_sounds"].append(sound_path)
    return LLM_STORY_, TL(text, to_code=language), TL(chapter, to_code=language), translate_decisions(decisions, TL, language), image_path, sound_path, Params

In [ ]:
def next_gen(Params, LLM_STORY_, LLM_PURE_, chap_num, decision):
    LLM_STORY_, chapter, text = LLM_story(LLM_STORY_, f"Write now the {ordinal(chap_num)} chapter of {params['num_chapters']} with using decision {decision}.", "story", LLM_PURE_) # type: ignore
    LLM_STORY_, decisions = LLM_story(LLM_STORY_, f"Create now {Params['num_decisions']} decisions how the story could continue.", "decisions", LLM_PURE_) # type: ignore
    
    image_path = SDXL(SDXL_, DETECTOR_, LLM_PURE_, text, chap_num, decision)
    sound_path = TTM(TTM_, LLM_PURE_, text, chap_num, decision, Params["prev_sounds"], Params["use_ai_music"])
    
    language = Params["language"]
    Params["prev_sounds"].append(sound_path)
    # Keep only the last two items
    if len(Params["prev_sounds"]) > 2:
        Params["prev_sounds"] = Params["prev_sounds"][-2:]
    return LLM_STORY_, TL(text, to_code=language), TL(chapter, to_code=language), translate_decisions(decisions, TL, language), image_path, sound_path, Params

In [ ]:
def last_gen(Params, LLM_STORY_, LLM_PURE_, chap_num, decision):
    LLM_STORY_, chapter, text = LLM_story(LLM_STORY_, f"Write now the {ordinal(chap_num)} chapter of {params['num_chapters']} with using decision {decision}. Find a happy or sad ending based on the previous decisions.", "story", LLM_PURE_) # type: ignore

    image_path = SDXL(SDXL_, DETECTOR_, LLM_PURE_, text, chap_num, decision)
    sound_path = TTM(TTM_, LLM_PURE_, text, chap_num, decision, Params["prev_sounds"], Params["use_ai_music"])
    
    language = Params["language"]
    Params["prev_sounds"].append(sound_path)
    # Keep only the last two items
    if len(Params["prev_sounds"]) > 2:
        Params["prev_sounds"] = Params["prev_sounds"][-2:]
    return LLM_STORY_, TL(text, to_code=language), TL(chapter, to_code=language), image_path, sound_path, Params

In [ ]:
def con_gen(Params, LLM_STORY_, LLM_PURE_):
    LLM_STORY_, chapter, text = LLM_story(LLM_STORY_, f"Write now a detailed text to evaluate the decisions of {params['main_name']}, and show what was good or bad and why! Please call {params['main_name']} by you. Write as chapter 'Reflections of your Adventure'", "story", LLM_PURE_) # type: ignore

    sound_path = TTM(TTM_, LLM_PURE_, text, Params["num_chapters"]+1, 1, Params["prev_sounds"], Params["use_ai_music"])
    
    language = Params["language"]
    Params["prev_sounds"].append(sound_path)
    # Keep only the last two items
    if len(Params["prev_sounds"]) > 2:
        Params["prev_sounds"] = Params["prev_sounds"][-2:]

    return TL(text, to_code=language), TL(chapter, to_code=language), sound_path, Params

In [ ]:
params = {}
params["main_name"] = "Klaus"
params["gender"] = "male"
params["language"] = "en"
params["country"] = "south korea"
params["colour"] = "blue"
params["animal"] = "crocodile"
params["animal_name"] = "Chris P Bacon"
params["season"] = "winter"
params["num_decisions"] = 4
params["num_chapters"] = 3
params["use_ai_music"] = True
params["prev_sounds"] = []

LLM_STORY_, cur_story, chapter_title, decisions, image_path, sound_path, params = start_gen(params, LLM_PURE_)

In [ ]:
print(chapter_title)
print(cur_story)
print(decisions)
print(image_path)
print(sound_path)
print(params)

In [ ]:
LLM_STORY_, cur_story, chapter_title, decisions, image_path, sound_path, params = next_gen(params, LLM_STORY_, LLM_PURE_, 2, 3)

In [ ]:
print(chapter_title)
print(cur_story)
print(decisions)
print(image_path)
print(sound_path)
print(params)

In [ ]:
LLM_STORY_, cur_story, chapter_title, image_path, sound_path, params = last_gen(params, LLM_STORY_, LLM_PURE_, 3, 2)

In [ ]:
print(chapter_title)
print(cur_story)
print(image_path)
print(sound_path)
print(params)

In [ ]:
cur_story, chapter_title, sound_path, params = con_gen(params, LLM_STORY_, LLM_PURE_)

In [ ]:
print(chapter_title)
print(cur_story)
print(sound_path)
print(params)

In [ ]:
LLM_STORY_.memory.buffer